In [1]:
import os
import pandas as pd

### merge.csv

In [2]:
def read_csv(file_path):
    if not os.path.exists(file_path):
        print("Error: File not found.")
        return None
    else:
        df = pd.read_csv(file_path)
        return df

In [3]:
def save_csv(dataframe, file_path):
    try:
        dataframe.to_csv(file_path, index=False)
        print("CSV file saved successfully.")
    except Exception as e:
        print(f"Error saving CSV file: {e}")

In [4]:
def delete_0_status(data_frame, column_name="Status"):
    try:
        data_frame = data_frame[data_frame[column_name] != 0]
        return data_frame
    except Exception as e:
        print(f"Error deleting rows with status zero: {e}")
        return None


In [8]:
def merge(file1, file2):
    df1 = read_csv(file1) 
    df2 = read_csv(file2)

    same_rows = pd.merge(df1, df2, how="inner")
    print("same rows:", same_rows)

    merged_df = pd.merge(df1, df2, on=list(df1.columns), how='outer')
    # show duplicated row
    duplicates = merged_df[merged_df.duplicated()]
    if not duplicates.empty:
        print(duplicates)
    else:
        print("no duplicate row")
        
    # drop duplicata rows
    merged_df.drop_duplicates(inplace=True, keep="first")
    
    return merged_df

In [9]:
def type_reform(df, column_name="Type of Report"):
    # construct report_type: tcfd -> 1, none_tcfd->0

    new_column_name = "tcfd"
    df[column_name] = df[column_name].fillna("") # del nan 
    df[new_column_name] = df[column_name].apply(lambda x: 1 if x == 'TCFD' else (0 if 'TCFD' not in x else -1))
    
    # delete status == 0 row
    df = delete_0_status(df, column_name="Status")

    return df

In [11]:
if __name__ == "__main__":
    # merge two files
    esg = "./input/esg_new.csv"    # (123, 7)
    tcfd = "./input/tcfd_new.csv"  # (118, 7)
    merge_df = merge(esg, tcfd)

    merged_file = "./output/merge.csv" # (187, 8)
    reform_df = type_reform(merge_df)
    save_csv(reform_df, merged_file)

same rows:      Company                                         Report URL  \
0  Wolfspeed  https://assets.wolfspeed.com/uploads/2022/10/W...   
1    Aurizon          https://www.aurizon.com.au/sustainability   

          Type of Report  Year Published  \
0  TCFD,SASB,GRI,UN_SDGs            2022   
1                   TCFD            2020   

                                       Industry                 Geography  \
0  Energy Equipment and Services, Manufacturing  United States of America   
1                           Rail Transportation                 Australia   

   Status  
0       1  
1       1  
            Company                                         Report URL  \
80       Fortum Oyj  https://www.fortum.com/about-us/investors/repo...   
81       Fortum Oyj  https://www.fortum.com/about-us/investors/repo...   
117  AcBel Polytech    https://www.acbel.com.tw/en/csr-report-download   

    Type of Report  Year Published                    Industry Geography  \
80       TCFD

### save merge_onehot.csv

In [12]:
def one_hot(df, column_name="Type of Report"):
    # construct report_type: tcfd -> 1, none_tcfd->0
    # df = pd.read_csv(input_file)

    column_data = df[column_name]

    # split ","
    split_contents = set()
    for item in column_data.str.split(','):
        if isinstance(item, list):
            split_contents.update(filter(None, map(str.strip, item)))

    print(split_contents)
    for content in split_contents:
        new_column_name = f"{content.strip()}"
        print("new_column_name: ", new_column_name)
        df[new_column_name] = df.apply(lambda row: 1 if isinstance(row[column_name], str) and new_column_name in [x.strip() for x in row[column_name].split(',')] else 0, axis=1)
    
    # delete status == 0 row
    df = delete_0_status(df, column_name="Status")

    return df

In [14]:
if __name__ == "__main__":
    # merge files
    esg = "./input/esg_new.csv"    # (123, 7)
    tcfd = "./input/tcfd_new.csv"  # (118, 7)
    merge_df2 = merge(esg, tcfd)
    
    # process merge file
    final_df = one_hot(merge_df2)
    merged_file = "./output/merge_onehot.csv" # (187, )
    save_csv(final_df, merged_file)

same rows:      Company                                         Report URL  \
0  Wolfspeed  https://assets.wolfspeed.com/uploads/2022/10/W...   
1    Aurizon          https://www.aurizon.com.au/sustainability   

          Type of Report  Year Published  \
0  TCFD,SASB,GRI,UN_SDGs            2022   
1                   TCFD            2020   

                                       Industry                 Geography  \
0  Energy Equipment and Services, Manufacturing  United States of America   
1                           Rail Transportation                 Australia   

   Status  
0       1  
1       1  
            Company                                         Report URL  \
80       Fortum Oyj  https://www.fortum.com/about-us/investors/repo...   
81       Fortum Oyj  https://www.fortum.com/about-us/investors/repo...   
117  AcBel Polytech    https://www.acbel.com.tw/en/csr-report-download   

    Type of Report  Year Published                    Industry Geography  \
80       TCFD